In [1]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time
import datetime

In [2]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'BOG'
    
#year-month-day
todays_date = datetime.datetime.today().strftime('%Y-%m-%d')
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = (datetime.datetime.today() - datetime.timedelta(days=30)).strftime('%Y-%m-%d') #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

dias = 0

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

### Loading COSTUMERS INFO

In [3]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",118629,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [4]:
df_margin = get_fresh_query_result("https://internal-redash.federate.frubana.com/",144521,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [5]:
df_hooks = get_fresh_query_result("https://internal-redash.federate.frubana.com/",108977,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


In [6]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [7]:
data_ventas_IQ = data_ventas.groupby(by=['region_code','customer_id','subcat']).agg({"gmv_usd":np.sum, "order_id":pd.Series.nunique}).reset_index()
data_ventas_IQ["AOV"] = data_ventas.gmv_usd/data_ventas.order_id

In [8]:
data_ventas = data_ventas.groupby(by=['region_code','customer_id']).agg({"gmv_usd":np.sum, "order_id":pd.Series.nunique}).reset_index()
data_ventas["AOV"] = data_ventas.gmv_usd/data_ventas.order_id

In [9]:
query = """
-- 93058
-- Se toma como base el query 2718
WITH 
ful AS (
SELECT DISTINCT
    bo.order_id,
    bo.submit_date
    
FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_group" bfg        ON bfg.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_fulfillment_order" bfo        ON bfo.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_fulfillment_group" ffg         ON ffg.fulfillment_group_id = bfg.fulfillment_group_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order" fo                      ON fo.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_payment" bop            ON bop.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site" s                       ON s.site_id = bo.site_disc
LEFT JOIN  postgres_broadleaf_federate."broadleaf.fb_order_type" fot                ON fot.fb_order_type_id=fo.fb_order_type_id

WHERE fo.fb_order_status_id IN (1,6,7,8)
    AND bo.submit_date >= DATE_TRUNC('day', date(getdate()) - interval '30 day')
    AND bo.order_status = 'SUBMITTED'
    AND bfo.status NOT IN ('ARCHIVED','CANCELLED')
    AND bop.archived = 'N'
    AND (fot.name IS NULL OR fot.name <> 'REFUND')
    AND s.site_identifier_value = '{city}'
),

view_sku AS (
SELECT DISTINCT
  --ds.identifier_value,
  s.site_identifier_value,
  --ea.ctsessionid,
  ea.profile_identity,
  ea.frproductid,
  bs2.name,
  bcat2.name AS category,
  bcat.name AS subcat,
  --ea.frproductname,
  --ea.frproductinstock,
  DATE_TRUNC('day', ea.event_date) AS event_date
  --ea.eventname,
  --ea.frrecommendationtype,
  --ea.frfromproductid
  --ea.frscreen--,
  --ea.frsubmitresult--,--para el cart add
  --info_subs.num_substitutes
FROM dpr_clevertap.events_analytics                                     ea
INNER JOIN dpr_clevertap.user_profile                                   p       ON ea.profile_identity = p.profile_identity AND ea.profile_identity NOT ILIKE '%,%'
INNER JOIN dpr_shared.dim_site                                          ds      ON ds.identifier_value = p.registeredcity
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"          bp      ON bp.product_id::varchar = ea.frproductid::varchar
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"             s       ON s.site_id = bp.catalog_disc AND ds.identifier_value = s.site_identifier_value
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"              bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"         bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"    bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"         bcat2   ON bcx.category_id = bcat2.category_id

WHERE
  ea.eventname = 'FRF Product View'
  AND ea.event_date > current_date - 30
  AND ds.identifier_value = '{city}'
 ),
 
almost AS (

SELECT DISTINCT
    s.site_identifier_value AS city,
    bo.customer_id,
    ful.submit_date,
    --bs2.name,
    bs.addl_product_id as padre_sku_id
    --bcat2.name AS category,
    --bcat.name AS subcat,
    -- CASE WHEN s.site_identifier_value = 'CMX' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'GDL' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'PBC' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'MTY' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'SPO' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'BHZ' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'CWB' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'VCP' then SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/4.75)
    -- ELSE SUM ( (boi.quantity * foi.step_unit * boi.sale_price)*1.0/3776) END AS gmv_usd,
    -- CASE WHEN s.site_identifier_value = 'CMX' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'GDL' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'PBC' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'MTY' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/19.65)
    --     WHEN s.site_identifier_value = 'SPO' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'BHZ' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'CWB' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    --     WHEN s.site_identifier_value = 'VCP' then (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/4.75)
    -- ELSE (SUM(boida.adjustment_value*boipd.quantity*foi.step_unit)*1.0/3776) END AS discount_usd,
    -- SUM(boi.quantity* foi.step_unit) as cant

FROM postgres_broadleaf_federate."broadleaf.blc_order" bo
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_site"                     s       ON s.site_id = bo.site_disc
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item"               boi     ON boi.order_id=bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order_item"                foi     ON boi.order_item_id= foi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.fb_order"                     fo      ON fo.order_id = bo.order_id
INNER JOIN ful                                                                          ON ful.order_id = bo.order_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_discrete_order_item"      bdoi    ON bdoi.order_item_id = boi.order_item_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs      ON bs.sku_id = bdoi.sku_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_product"                  bp      ON bs.addl_product_id = bp.product_id
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_sku"                      bs2     ON bs2.sku_id = bp.default_sku_id --Conectar la tarjeta con el sku que guarda la info
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat    ON bcat.category_id = bp.default_category_id 
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category_xref"            bcx     ON bcx.sub_category_id = bp.default_category_id AND bcx.archived='N' AND bcx.sndbx_tier is NULL
INNER JOIN postgres_broadleaf_federate."broadleaf.blc_category"                 bcat2   ON bcx.category_id = bcat2.category_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_price_dtl"     boipd   ON boipd.order_item_id=boi.order_item_id
--INNER JOIN postgres_broadleaf_federate."broadleaf.blc_order_item_dtl_adj"       boida   ON boida.order_item_price_dtl_id=boipd.order_item_price_dtl_id

WHERE fb_order_status_id IN (1,6,7,8)
    AND bo.order_status = 'SUBMITTED'
   -- FIX SUPER DESCUENTOS
    AND bcat2.category_id not in ('110873','-1000','100768','100765','100815') --ids de super descuentos en cada país
    AND bcat.name <> 'Oferton Frubana' AND bcat2.name <> 'Oferton Frubana'
    AND bo.customer_id IN (SELECT DISTINCT profile_identity FROM view_sku)
)

SELECT DISTINCT
    vs.site_identifier_value AS city,
    vs.profile_identity AS customer_id,
    vs.frproductid AS card_id,
    vs.name AS product,
    vs.category AS cat,
    CASE
        WHEN vs.subcat ILIKE 'Pollo%' THEN 'Pollo'
        WHEN vs.subcat ILIKE 'Res%' THEN 'Res'
        WHEN vs.subcat ILIKE 'Cerdo%' THEN 'Cerdo'
        ELSE vs.subcat
    END AS subcat

FROM view_sku vs
LEFT JOIN almost a ON a.customer_id = vs.profile_identity 
 AND a.city = vs.site_identifier_value 
 AND a.padre_sku_id = vs.frproductid 
 AND a.submit_date BETWEEN vs.event_date AND DATEADD(day, 1, vs.event_date)
-- AND (a.submit_date = vs.event_date OR a.submit_date = DATEADD(day, 5, vs.event_date))

WHERE a.customer_id IS NULL
""".format(city = city_code)

df_view = read_connection_data_warehouse.run_read_prod_query(query)
df_view[["customer_id", "card_id"]] = df_view[["customer_id", "card_id"]].astype(int)

In [10]:
from sklearn.model_selection import train_test_split 

def create_groups(offer, users):
    
    #customers = offer.customer_id.unique()
    customers = np.union1d(offer.customer_id.unique(), users)
    print("Total length: ", len(customers))
    customer_control, customer_test = train_test_split(customers, test_size = 0.80)
    
    alpha=0.01
    aux=0
    while abs(1-(offer[offer.customer_id.isin(customer_control)]['AOV'].mean()/offer[offer.customer_id.isin(customer_test)]['AOV'].mean())) > alpha:
        customer_control, customer_test = train_test_split(customers, test_size = 0.80)
        
        aux+=1
        if aux == 1000:
            alpha+=0.01
            aux=0
        
    return customer_control.tolist(), customer_test.tolist()

In [11]:
def list_users(dfIQ, rangos):
    """This function select the customer_ids of the clients who are between the range of gmv_usd"""
    total_users = dfIQ.customer_id.unique().tolist()
    
    #print(f'Total users: {len(total_users)}')
    
    inferior, superior = np.percentile(dfIQ['gmv_usd'],rangos) #rango intercuartil entre los skus a tomar
    
    users = dfIQ[(dfIQ.gmv_usd>=inferior) & (dfIQ.gmv_usd<=superior)].groupby(['customer_id']).gmv_usd.sum().reset_index()
    print(f'Total users: {len(users)}')
    
    return users.customer_id.unique()

In [12]:
def get_info(df, sku,rango):
    users = list_users(data_ventas_IQ.loc[data_ventas_IQ.subcat == df_view.loc[df_view.card_id == sku,'subcat'].min()],rango)
    control, test = create_groups(data_ventas.loc[data_ventas.customer_id.isin(df.customer_id.unique())],users)
    name = df["product"].unique()[0].replace(",", ".")
    
    print(name+": \nCONTROL: ",len(control)," PILOTO: ",len(test))
    print("MARGEN: ", df_margin.loc[(df_margin.id_tarjeta == sku) & (df_margin.day == df_margin.day.max()),"margin"].values[0].round(3)*100)
    
    return control, test, name

In [13]:
"""Here we should complete our data for each SH, important if we want to submit a subcategory SH offer, in SKUS we should put the skus into a list as this [[1,2,3]]"""
card_id = [40274,413529]
max_uses = [30,30]
max_orders = [2,2]
rango_IQ = [[0,50],[0,50]]

In [19]:
offer_name = []
global_segment_list = [] #//for procesing
hooks = []
for j, sku in enumerate(card_id):
    if len(df_view.loc[df_view.card_id == sku]) != 0:
        print(len(df_view.loc[df_view.card_id == sku]))
        control,test,name = get_info(df_view.loc[df_view.card_id == sku], sku,rango_IQ[j])

        offer_name.append("SH_TRES_"+todays_date+"_"+city_code+"_"+name)
        d = {'name':"SH_TRES_"+todays_date+"_"+city_code+"_"+name, 'customersIds': list(test),"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(dias+7))).strftime('%Y-%m-%d')}
        e = {'name':"SH_TRES_"+todays_date+"_"+city_code+"_"+name+"_CONTROL", 'customersIds': list(control),"expiresAt": (datetime.datetime.today() + datetime.timedelta(days=(dias+7))).strftime('%Y-%m-%d')}
        
        global_segment_list.append(d)
        global_segment_list.append(e)
        
        if df_hooks.loc[df_hooks.addl_product_id == sku].value_discount.max():
            hooks.append(df_hooks.loc[df_hooks.addl_product_id == sku].value_discount.max())
        else:
            hooks.append(0)
    else:
        print("0 BUSQUEDAS")

653
Total users: 1261
Total length:  1501
Chocolate La Especial x 500gr: 
CONTROL:  300  PILOTO:  1201
MARGEN:  6.1
577
Total users: 2926
Total length:  3271
Contenedor J1 Darnel x 200 und: 
CONTROL:  654  PILOTO:  2617
MARGEN:  -0.5


In [20]:
discount = [8,5]

In [21]:
offer_name2 = []
for i, name in enumerate(offer_name):
    # Retrieve rows that match the condition
    matching_rows = df_hooks.loc[df_hooks.addl_product_id == card_id[i], ["value_discount"]]
    
    # Check if matching_rows is empty
    if matching_rows.empty:
        default_value = 0
    else:
        # Get the unique discount value from matching rows
        discount_value = matching_rows["value_discount"].unique()[0]
        
        # Calculate the updated value
        default_value = round(discount_value / discount[i], 2)
    
    # Update offer_name[i] with the calculated value
    offer_name2.append(offer_name[i] + "_HDCT_" + str(default_value))    
print(offer_name2)

['SH_TRES_2023-09-15_BOG_Chocolate La Especial x 500gr_HDCT_0.38', 'SH_TRES_2023-09-15_BOG_Contenedor J1 Darnel x 200 und_HDCT_0']


In [22]:
print(offer_name)
print(offer_name2)

['SH_TRES_2023-09-15_BOG_Chocolate La Especial x 500gr', 'SH_TRES_2023-09-15_BOG_Contenedor J1 Darnel x 200 und']
['SH_TRES_2023-09-15_BOG_Chocolate La Especial x 500gr_HDCT_0.38', 'SH_TRES_2023-09-15_BOG_Contenedor J1 Darnel x 200 und_HDCT_0']


In [25]:
csv_file = {} 
csv_file['offer_name'] = offer_name2
csv_file['offer_description'] = None
csv_file['discount'] = discount
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = (datetime.datetime.today() + datetime.timedelta(days=dias)).strftime('%d-%m-%Y')
csv_file['end_date'] = (datetime.datetime.today() + datetime.timedelta(days=(dias+7))).strftime('%d-%m-%Y')
csv_file['max_uses_per_order'] = max_uses
csv_file['max_uses_per_customer'] = max_orders
csv_file['customer_segment_id'] = [1]*len(offer_name) #[None]
csv_file['sku_id'] = [frida_products.loc[frida_products.id_tarjeta == sku,"skus"].values[0] for sku in card_id]

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [26]:
offer_name2

['SH_TRES_2023-09-15_BOG_Chocolate La Especial x 500gr_HDCT_0.38',
 'SH_TRES_2023-09-15_BOG_Contenedor J1 Darnel x 200 und_HDCT_0']

In [27]:
import requests
import json

url = f"https://{city_code}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

payload = json.dumps(global_segment_list)


headers = {
  'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
  'Content-Type': 'application/json'
}

response = requests.request("POST", url, headers=headers, data=payload)

In [29]:
sgt = [(int(x[:6]),x[7:]) for x in list(map(str.strip, response.text[32:].strip('][').replace('"', '').split(',')))]

In [30]:
sgt

[(351952, 'SH_TRES_2023-09-15_BOG_Chocolate La Especial x 500gr'),
 (351951, 'SH_TRES_2023-09-15_BOG_Chocolate La Especial x 500gr_CONTROL'),
 (351954, 'SH_TRES_2023-09-15_BOG_Contenedor J1 Darnel x 200 und'),
 (351953, 'SH_TRES_2023-09-15_BOG_Contenedor J1 Darnel x 200 und_CONTROL')]

In [31]:
for i,name in enumerate(global_offer_csv.offer_name):
    for j,z in sgt:
        if z==offer_name[i]:
            global_offer_csv.loc[[i], ('customer_segment_id')] = j
            break

In [32]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_TRES_2023-09-15_BOG_Chocolate La Especial x...,None,8,true,15-09-2023,22-09-2023,30,2,351952,47547
1,SH_TRES_2023-09-15_BOG_Contenedor J1 Darnel x ...,None,5,true,15-09-2023,22-09-2023,30,2,351954,575830


In [29]:
global_offer_csv["offer_name"] = [f"{row['offer_name']}_HDCT_{hook/row['discount']:.2f}" for hook, (_, row) in zip(hooks, global_offer_csv.iterrows())]

In [30]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_TRES_2023-09-11_BOG_Chocolate La Especial x...,None,8,true,11-09-2023,18-09-2023,30,2,342238,47547
1,SH_TRES_2023-09-11_BOG_Contenedor J1 Darnel x ...,None,5,true,11-09-2023,18-09-2023,30,2,342239,575830


In [61]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/Below_Media/{city_code}/{todays_date}_SH_PLUS_{city_code}_OFFER_csv.csv', index = False)